В качестве генератора случайных чисел выберем когруэнтный метод из прошлой лаб раб 

In [10]:
import math
def congruent_method_generator():
    m =10**10
    x=get_x0(m)
    a=197
    while True:
        x = a*x%m
        yield x * 1.0 / m


def get_x0(m):
    for i in range(m-1,1,-1):
        if math.gcd(i,m) == 1:
            return i
    return 1

In [8]:
class RandomEventSimulator:

    def __init__(self):
        self._generator = congruent_method_generator()
    
    def check_event(self,p):
        _p = next(self._generator)
        return _p < p
    

In [11]:
p_list = [0.5, 0.1, 0.8]
N_SAMPLES = 10**6
simulator = RandomEventSimulator()


for p in p_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_event(p))
    happend_event_count = sum(answers)
    print(f"p: {p} happend events count :{happend_event_count}")


p: 0.5 happend events count :500247
p: 0.1 happend events count :100224
p: 0.8 happend events count :800504


In [16]:
import numpy as np
class NotSimpleEventSimulator():

    def __init__(self):
        self._generator = congruent_method_generator()
    
    def _first(self,it,condition):
        return next(x for x in it if condition(x))
    def check_event(self,k):
        p = next(self._generator)
        _cumsum = enumerate(np.cumsum(k))
        success_event = self._first(_cumsum, lambda x : p <=x[1])[0]
        bool_vector = [False for _ in k]
        bool_vector[success_event] = True
        return bool_vector

N_SAMPLES = 10**6

k_list=[
    [0.1,0.2,0.3,0.4],
    [0.8,0.1,0.05,0.05],
    [0.5,0.5]
]

simulator = NotSimpleEventSimulator()

for k in k_list:
    answers = list()
    for i in range(N_SAMPLES):
        answers.append(simulator.check_event(k))
    event_freq = [0 for i in k]
    for answer in answers:
        for i in range(len(k)):
            event_freq[i]+= answer[i]
    print(f"event freq:{event_freq}")



event freq:[0, 0, 0, 1]
